In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**Brief**

Write the architecture of a SimpleUNet following the diagram shown during the lecture/in GitHub:

Encoder: Define a convolutional block (conv_block) that will be repeated in each layer of the encoder. Define as well a maxpooling block (maxpool_block) that will also be repeated when necessary during the encoding (downsampling path). Hint: use torch.nn.Conv2d and remember you need to add by hand the activation function (use ReLU).

Middle: define the middle or bottleneck part of the architecture.

Decoder: Define the transposed convolutional blocks (transposed_block). Hint: use nn.ConvTranspose2d.

Final layer: Define the final convolutional layer.

Forward: Remember you need to define the forward pass of your class, with how each block will be called.

In order to aid the optimisation process, include batch normalisation (nn.BatchNorm2d) and 50% dropout (nn.Dropout2d(0.5)). Where do think they should be placed?

Note: There are different ways to build this architecture, here we will use PyTorch as we will test that we can load the pre-trained model provided.

In [18]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
# !pip install torchmetrics
import torchmetrics
from torchmetrics.classification import BinaryAccuracy
import numpy as np

from utils.makepath import makepath as mkp
from networks.mri_pdhg_net import MriPdhgNet
from scripts.mri.model_loader import ModelLoader

In [19]:
root_dir = mkp('..', '..')

u_tgv_model_path = mkp(root_dir, 'pretrained', 'u_tgv')

# Pretrained model file
u_tgv_file = 'u_tgv-model_state_dict_30-cpu.pth'

# Load the state_dict
state_dict = torch.load(
    mkp(u_tgv_model_path, u_tgv_file),
    map_location=torch.device('cpu'),
    weights_only=False
)

In [20]:
len(state_dict.keys())

37

In [21]:
list(state_dict.keys())[0:10]

['beta',
 'cnn.encoder.enc_blocks.0.conv_block.0.weight',
 'cnn.encoder.enc_blocks.0.conv_block.0.bias',
 'cnn.encoder.enc_blocks.0.conv_block.2.weight',
 'cnn.encoder.enc_blocks.0.conv_block.2.bias',
 'cnn.encoder.enc_blocks.1.conv_block.0.weight',
 'cnn.encoder.enc_blocks.1.conv_block.0.bias',
 'cnn.encoder.enc_blocks.1.conv_block.2.weight',
 'cnn.encoder.enc_blocks.1.conv_block.2.bias',
 'cnn.encoder.enc_blocks.2.conv_block.0.weight']

In [22]:
model_loader = ModelLoader(
    config_choice=mkp(root_dir, 'config', 'example_mri_tgv_config_test.yaml'),
    is_training=False
)

Config loaded from file ../../config/example_mri_tgv_config_test.yaml
Loading model on device: cpu


In [23]:
# Initialize a new model
model = model_loader.load_pretrained_model(
    root_dir=root_dir
)

Norm of operator A: 1
Norm of gradient operator nabla: 2.8284270763397217
L: 3.0
PDHG net device: cpu


RuntimeError: Error(s) in loading state_dict for MriPdhgNet:
	Missing key(s) in state_dict: "cnn.c0x0.conv_block.0.weight", "cnn.c0x0.conv_block.0.bias", "cnn.c0x0.conv_block.2.weight", "cnn.c0x0.conv_block.2.bias", "cnn.encoder.0.double_conv.conv_block.0.weight", "cnn.encoder.0.double_conv.conv_block.0.bias", "cnn.encoder.0.double_conv.conv_block.2.weight", "cnn.encoder.0.double_conv.conv_block.2.bias", "cnn.encoder.1.double_conv.conv_block.0.weight", "cnn.encoder.1.double_conv.conv_block.0.bias", "cnn.encoder.1.double_conv.conv_block.2.weight", "cnn.encoder.1.double_conv.conv_block.2.bias", "cnn.encoder.2.double_conv.conv_block.0.weight", "cnn.encoder.2.double_conv.conv_block.0.bias", "cnn.encoder.2.double_conv.conv_block.2.weight", "cnn.encoder.2.double_conv.conv_block.2.bias", "cnn.encoder.3.double_conv.conv_block.0.weight", "cnn.encoder.3.double_conv.conv_block.0.bias", "cnn.encoder.3.double_conv.conv_block.2.weight", "cnn.encoder.3.double_conv.conv_block.2.bias", "cnn.decoder.0.u

In [52]:
model.cnn

UNet(
  (encoder): Encoder(
    (enc_blocks): ModuleList(
      (0): ConvBlock(
        (conv_block): Sequential(
          (0): Conv2d(2, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): LeakyReLU(negative_slope=0.01)
          (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (3): LeakyReLU(negative_slope=0.01)
        )
      )
      (1): ConvBlock(
        (conv_block): Sequential(
          (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): LeakyReLU(negative_slope=0.01)
          (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (3): LeakyReLU(negative_slope=0.01)
        )
      )
      (2): ConvBlock(
        (conv_block): Sequential(
          (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): LeakyReLU(negative_slope=0.01)
          (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (3

In [53]:
for block_idx in [1, 2, 3]:
    expected_shape = state_dict[f'conv{block_idx}.0.weight'].shape
    actual_shape = model.__getattr__(f'conv{block_idx}')[0].weight.shape
    assert expected_shape == actual_shape, \
        f"Convo block {block_idx} weight shape mismatch: expected {expected_shape}, got {actual_shape}"

KeyError: 'conv1.0.weight'

In [84]:
# Load the model
model.load_state_dict(
    state_dict,
    strict=False
)
model.eval()

SimpleUNet(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Dropout2d(p=0.5, inplace=False)
  )
  (maxpool1): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Dropout